#**Importing and forming the datset**

In [ ]:
import pandas as pd

df = pd.read_json('data.json')

dataset = {'title':[], 'paragraph': [], 'context': [], 'question': [], 'answer': [], 'answer_start': []}
for i in range(df['data'].shape[0]):
  paragraph_count = 0
  for j in range(len(df['data'][i]['paragraphs'])):
    for k in range(len(df['data'][i]['paragraphs'][j]['qas'])):
      for l in range(len(df['data'][i]['paragraphs'][j]['qas'][k]['answers'])):
        dataset['title'].append(df['data'][i]['title'])
        dataset['paragraph'].append(paragraph_count)
        dataset['context'].append(df['data'][i]['paragraphs'][j]['context'])
        dataset['question'].append(df['data'][i]['paragraphs'][j]['qas'][k]['question'])
        dataset['answer'].append(df['data'][i]['paragraphs'][j]['qas'][k]['answers'][l]['text'])
        dataset['answer_start'].append(df['data'][i]['paragraphs'][j]['qas'][k]['answers'][l]['answer_start'])
    paragraph_count += 1

data = pd.DataFrame.from_dict(dataset)
data.head(15)

,title,paragraph,context,question,answer,answer_start
0,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,515
1,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,188
2,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,279
3,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,381
4,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,92
5,University_of_Notre_Dame,1,"As at most other universities, Notre Dame's st...",When did the Scholastic Magazine of Notre dame...,September 1876,248
6,University_of_Notre_Dame,1,"As at most other universities, Notre Dame's st...",How often is Notre Dame's the Juggler published?,twice,441
7,University_of_Notre_Dame,1,"As at most other universities, Notre Dame's st...",What is the daily student paper at Notre Dame ...,The Observer,598
8,University_of_Notre_Dame,1,"As at most other universities, Notre Dame's st...",How many student news papers are found at Notr...,three,126
9,University_of_Notre_Dame,1,"As at most other universities, Notre Dame's st...",In what year did the student paper Common Sens...,1987,908


#**BERT QnA large uncased pre-trained model**

#**Doing the necessary imports**

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForQuestionAnswering

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = TFBertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

All the weights of TFBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [ ]:
def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    # Report how long the input sequence is.
    print('No. of tokens (Question + Answer): ', len(input_ids))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token itself.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Convert inputs to tensors
    input_ids_tensor = tf.convert_to_tensor([input_ids])
    segment_ids_tensor = tf.convert_to_tensor([segment_ids])

    # Run the example through the model.
    outputs = model(input_ids_tensor, token_type_ids=segment_ids_tensor, return_dict=True)

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = tf.argmax(start_scores, axis=1).numpy()[0]
    answer_end = tf.argmax(end_scores, axis=1).numpy()[0]

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):

        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]

        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]
    print('Question: "'+ question + '"')
    print('Answer: "'+ answer + '"')
    print()

# Example usage:
for i in range(5):
  question = data['question'].iloc[i]
  context = data['context'].iloc[i]
  answer_question(question, context)

No. of tokens (Question + Answer):  176
Question: "To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?"
Answer: "saint bernadette soubirous"

No. of tokens (Question + Answer):  172
Question: "What is in front of the Notre Dame Main Building?"
Answer: "a copper statue of christ"

No. of tokens (Question + Answer):  176
Question: "The Basilica of the Sacred heart at Notre Dame is beside to which structure?"
Answer: "the main building"

No. of tokens (Question + Answer):  170
Question: "What is the Grotto at Notre Dame?"
Answer: "a replica of the grotto at lourdes , france"

No. of tokens (Question + Answer):  173
Question: "What sits on top of the Main Building at Notre Dame?"
Answer: "a golden statue of the virgin mary"

